In [1]:
directory="/mnt/Personal/Repository/Raiden-Encrypted"
key_path="/mnt/Personal/Projects/Micromouse/KEYS"

In [2]:
import os
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA

def decrypt_file(input_file, output_file, private_key):
    with open(input_file, "rb") as f:
        enc_aes_key = f.read(256)  
        nonce = f.read(16)  
        tag = f.read(16)  
        ciphertext = f.read()

    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(private_key))
    aes_key = cipher_rsa.decrypt(enc_aes_key)

    cipher_aes = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher_aes.decrypt_and_verify(ciphertext, tag)

    with open(output_file, "wb") as f:
        f.write(plaintext)

def decrypt_directory(encrypted_dir):
    if not encrypted_dir.endswith("-Encrypted"):
        print("Invalid directory. Must end with '-Encrypted'.")
        return

    decrypted_dir = encrypted_dir.replace("-Encrypted", "-Decrypted")
    os.makedirs(decrypted_dir, exist_ok=True)

    private_key_path = os.path.join(key_path, "private_key.pem")
    if not os.path.exists(private_key_path):
        print("Private key not found. Decryption aborted.")
        return

    with open(private_key_path, "rb") as key_file:
        private_key = key_file.read()

    for root, dirs, files in os.walk(encrypted_dir):
        rel_path = os.path.relpath(root, encrypted_dir)
        new_root = os.path.join(decrypted_dir, rel_path)
        os.makedirs(new_root, exist_ok=True)

        for file in files:
            if not file.endswith(".enc"):
                continue
            input_path = os.path.join(root, file)
            output_path = os.path.join(new_root, file[:-4])  
            decrypt_file(input_path, output_path, private_key)

    print(f"Decryption complete. Decrypted files stored in {decrypted_dir}")

In [3]:
decrypt_directory(directory)

Decryption complete. Decrypted files stored in /mnt/Personal/Repository/Raiden-Decrypted


In [4]:
import os
import filecmp

def are_directories_equal(dir1, dir2):
    if not os.path.exists(dir1) or not os.path.exists(dir2):
        print("One or both directories do not exist.")
        return False

    comparison = filecmp.dircmp(dir1, dir2)

    if comparison.left_only:
        print(f"Files only in {dir1}: {comparison.left_only}")
    if comparison.right_only:
        print(f"Files only in {dir2}: {comparison.right_only}")
    if comparison.diff_files:
        for diff_file in comparison.diff_files:
            print(f"Files differ: {os.path.join(dir1, diff_file)} and {os.path.join(dir2, diff_file)}")

    # Recursively check subdirectories
    for subdir in comparison.common_dirs:
        subdir1 = os.path.join(dir1, subdir)
        subdir2 = os.path.join(dir2, subdir)
        are_directories_equal(subdir1, subdir2)

    # If there were any differences, return False
    return not (comparison.left_only or comparison.right_only or comparison.diff_files)

# Example usage:
dir1 = directory.replace("-Encrypted", "")
dir2 = directory.replace("-Encrypted", "-Decrypted")

if are_directories_equal(dir1, dir2):
    print("Directories are completely identical!")
else:
    print("Directories are different!")


Directories are completely identical!
